In [1]:
import keras
import numpy as np
import tensorflow as tf

from absl import flags
from pathlib import Path
from operator import itemgetter
from keras.optimizers import Adam
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

Using TensorFlow backend.


In [2]:
import importlib

import model as nn
importlib.reload(nn)

elsa_architecture = nn.elsa_architecture

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

lang = "ja"
batch_size = 250
lr = 1e-3
epochs = 100
patience = 3
data_dir = "/data/elsa2"
checkpoint_dir = "./ckpt"
optimizer = "adam"

In [4]:
lstm_hidden = 512
lstm_drop = 0.5
final_drop = 0.5
embed_drop = 0.0
highway = False
compute_class_weight = False
multilabel = True

In [5]:
data_dir = Path(data_dir)
wv_path = (data_dir / "{:s}_wv.npy".format(lang)).__str__()
X_path = (data_dir / "{:s}_X.npy".format(lang)).__str__()
y_path = (data_dir / "{:s}_y.npy".format(lang)).__str__()
emoji_path = (data_dir / "{:s}_emoji.txt".format(lang)).__str__()

wv = np.load(wv_path, allow_pickle=True)
input_vec = np.load(X_path, allow_pickle=True)
input_label = np.load(y_path, allow_pickle=True)

nb_tokens = len(wv)
embed_dim = wv.shape[1]
input_len = len(input_label)
nb_classes = input_label.shape[1]
maxlen = input_vec.shape[1]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

if multilabel:
    def to_multilabel(y):
        outputs = []
        for i in range(nb_classes):
            outputs.append(y[:, i])
        return outputs

    y_train = to_multilabel(y_train)
    y_val = to_multilabel(y_val)
    y_test = to_multilabel(y_test)

model = elsa_architecture(nb_classes=nb_classes,
                          nb_tokens=nb_tokens,
                          maxlen=maxlen,
                          final_dropout_rate=final_drop,
                          embed_dropout_rate=embed_drop,
                          load_embedding=True,
                          pre_embedding=wv,
                          high=highway,
                          embed_dim=embed_dim,
                          multilabel=multilabel)
model.summary()

computed_class_weight = None

if multilabel:
    loss = "binary_crossentropy"
else:
    loss = "categorical_crossentropy"
    if compute_class_weight:
        y_train_sps = []
        for row in y_train:
            y_train_sps.extend(np.where(row)[0].tolist())
        computed_class_weight = class_weight.compute_class_weight(
            'balanced', list(range(nb_classes)), y_train_sps)
        print("computed class weight = {:s}".format(str(computed_class_weight)))

if optimizer == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optimizer == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir.mkdir()
checkpoint_weight_path = (checkpoint_dir / "elsa_{:s}.hdf5".format(lang)).__str__()

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
    keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss',
                                    verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
]
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val),
          class_weight=computed_class_weight,
          callbacks=callbacks,
          verbose=1)

freq = {line.split()[0]: int(line.split()[1]) for line in open(emoji_path).readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

if multilabel:
    y_pred = model.predict([X_test], batch_size=batch_size)
    y_pred = [np.squeeze(p) for p in y_pred]

    y_test_1d = np.array(y_test).flatten()
    y_pred_1d = np.array(y_pred).flatten()
    print(f1_score(y_test_1d, y_pred_1d > 0.5))
    print(classification_report(y_test_1d, y_pred_1d > 0.5))

    gold, pred = [], []
    for i in range(len(X_test)):
        each_gold, each_pred = [], []
        for c in range(nb_classes):
            if y_test[c][i] == 1.0:
                each_gold.append(c+1)
            else:
                each_gold.append(0)
            if y_pred[c][i] > 0.5:
                each_pred.append(c+1)
            else:
                each_pred.append(0)
        gold.extend(each_gold)
        pred.extend(each_pred)

    target_name = [""] + [e[0] for e in freq_topn]
    print(classification_report(gold, pred, target_names=target_name))
else:
    _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(acc)

    y_pred = model.predict(X_test)
    print(classification_report(y_test.argmax(axis=1), y_pred.argmax(
        axis=1), target_names=[e[0] for e in freq_topn]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 200)      65831000    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 50, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1721192 samples, validate on 491770 samples
Epoch 1/100
1721192/1721192 [==============================] - 3589s 2ms/step - loss: 3.9125 - sigmoid_0_loss: 0.2159 - sigmoid_1_loss: 0.2036 - sigmoid_2_loss: 0.1918 - sigmoid_3_loss: 0.1273 - sigmoid_4_loss: 0.1349 - sigmoid_5_loss: 0.1621 - sigmoid_6_loss: 0.1459 - sigmoid_7_loss: 0.1048 - sigmoid_8_loss: 0.1074 - sigmoid_9_loss: 0.0815 - sigmoid_10_loss: 0.1072 - sigmoid_11_loss: 0.1154 - sigmoid_12_loss: 0.1032 - sigmoid_13_loss: 0.0640 - sigmoid_14_loss: 0.0420 - sigmoid_15_loss: 0.0823 - sigmoid_16_loss: 0.0769 - sigmoid_17_loss: 0.0786 - sigmoid_18_loss: 0.0708 - sigmoid_19_loss: 0.0807 - sigmoid_20_loss: 0.0752 - sigmoid_21_loss: 0.0580 - sigmoid_22_loss: 0.0559 - sigmoid_23_loss: 0.0458 - sigmoid_24_loss: 0.0630 - sigmoid_25_loss: 0.0560 - sigmoid_26_loss: 0.0581 - sigmoid_27_loss: 0.0492 - sigmoid_

    500/1721192 [..............................] - ETA: 56:33 - loss: 3.4123 - sigmoid_0_loss: 0.2136 - sigmoid_1_loss: 0.1471 - sigmoid_2_loss: 0.2003 - sigmoid_3_loss: 0.0901 - sigmoid_4_loss: 0.1574 - sigmoid_5_loss: 0.1286 - sigmoid_6_loss: 0.1128 - sigmoid_7_loss: 0.1309 - sigmoid_8_loss: 0.0964 - sigmoid_9_loss: 0.0957 - sigmoid_10_loss: 0.0748 - sigmoid_11_loss: 0.0869 - sigmoid_12_loss: 0.0993 - sigmoid_13_loss: 0.0772 - sigmoid_14_loss: 0.0270 - sigmoid_15_loss: 0.0535 - sigmoid_16_loss: 0.0642 - sigmoid_17_loss: 0.0457 - sigmoid_18_loss: 0.0585 - sigmoid_19_loss: 0.1083 - sigmoid_20_loss: 0.1172 - sigmoid_21_loss: 0.0202 - sigmoid_22_loss: 0.0263 - sigmoid_23_loss: 0.0285 - sigmoid_24_loss: 0.0322 - sigmoid_25_loss: 0.0336 - sigmoid_26_loss: 0.0658 - sigmoid_27_loss: 0.0394 - sigmoid_28_loss: 0.0588 - sigmoid_29_loss: 0.0459 - sigmoid_30_loss: 0.0364 - sigmoid_31_loss: 0.0363 - sigmoid_32_loss: 0.0175 - sigmoid_33_loss: 0.0074 - sigmoid_34_loss: 0.0029 - sigmoid_35_loss: 0.02

1721192/1721192 [==============================] - 3559s 2ms/step - loss: 3.6616 - sigmoid_0_loss: 0.1990 - sigmoid_1_loss: 0.1914 - sigmoid_2_loss: 0.1829 - sigmoid_3_loss: 0.1165 - sigmoid_4_loss: 0.1244 - sigmoid_5_loss: 0.1547 - sigmoid_6_loss: 0.1405 - sigmoid_7_loss: 0.0987 - sigmoid_8_loss: 0.0973 - sigmoid_9_loss: 0.0743 - sigmoid_10_loss: 0.1014 - sigmoid_11_loss: 0.1105 - sigmoid_12_loss: 0.0994 - sigmoid_13_loss: 0.0569 - sigmoid_14_loss: 0.0354 - sigmoid_15_loss: 0.0767 - sigmoid_16_loss: 0.0719 - sigmoid_17_loss: 0.0746 - sigmoid_18_loss: 0.0666 - sigmoid_19_loss: 0.0766 - sigmoid_20_loss: 0.0725 - sigmoid_21_loss: 0.0529 - sigmoid_22_loss: 0.0523 - sigmoid_23_loss: 0.0410 - sigmoid_24_loss: 0.0603 - sigmoid_25_loss: 0.0527 - sigmoid_26_loss: 0.0561 - sigmoid_27_loss: 0.0463 - sigmoid_28_loss: 0.0513 - sigmoid_29_loss: 0.0332 - sigmoid_30_loss: 0.0541 - sigmoid_31_loss: 0.0526 - sigmoid_32_loss: 0.0209 - sigmoid_33_loss: 0.0108 - sigmoid_34_loss: 0.0043 - sigmoid_35_loss: 

    500/1721192 [..............................] - ETA: 59:00 - loss: 3.5674 - sigmoid_0_loss: 0.2219 - sigmoid_1_loss: 0.1631 - sigmoid_2_loss: 0.1495 - sigmoid_3_loss: 0.1365 - sigmoid_4_loss: 0.1548 - sigmoid_5_loss: 0.1519 - sigmoid_6_loss: 0.1553 - sigmoid_7_loss: 0.0490 - sigmoid_8_loss: 0.1133 - sigmoid_9_loss: 0.0597 - sigmoid_10_loss: 0.1170 - sigmoid_11_loss: 0.0994 - sigmoid_12_loss: 0.0966 - sigmoid_13_loss: 0.0273 - sigmoid_14_loss: 0.0278 - sigmoid_15_loss: 0.0664 - sigmoid_16_loss: 0.0292 - sigmoid_17_loss: 0.1025 - sigmoid_18_loss: 0.0402 - sigmoid_19_loss: 0.0842 - sigmoid_20_loss: 0.0595 - sigmoid_21_loss: 0.0304 - sigmoid_22_loss: 0.0681 - sigmoid_23_loss: 0.0445 - sigmoid_24_loss: 0.0748 - sigmoid_25_loss: 0.0366 - sigmoid_26_loss: 0.0612 - sigmoid_27_loss: 0.0288 - sigmoid_28_loss: 0.0410 - sigmoid_29_loss: 0.0304 - sigmoid_30_loss: 0.0647 - sigmoid_31_loss: 0.0600 - sigmoid_32_loss: 0.0356 - sigmoid_33_loss: 0.0047 - sigmoid_34_loss: 7.3727e-04 - sigmoid_35_loss: 

1721192/1721192 [==============================] - 3550s 2ms/step - loss: 3.5881 - sigmoid_0_loss: 0.1934 - sigmoid_1_loss: 0.1872 - sigmoid_2_loss: 0.1798 - sigmoid_3_loss: 0.1132 - sigmoid_4_loss: 0.1204 - sigmoid_5_loss: 0.1517 - sigmoid_6_loss: 0.1381 - sigmoid_7_loss: 0.0962 - sigmoid_8_loss: 0.0936 - sigmoid_9_loss: 0.0715 - sigmoid_10_loss: 0.0992 - sigmoid_11_loss: 0.1083 - sigmoid_12_loss: 0.0979 - sigmoid_13_loss: 0.0547 - sigmoid_14_loss: 0.0338 - sigmoid_15_loss: 0.0751 - sigmoid_16_loss: 0.0700 - sigmoid_17_loss: 0.0729 - sigmoid_18_loss: 0.0651 - sigmoid_19_loss: 0.0749 - sigmoid_20_loss: 0.0716 - sigmoid_21_loss: 0.0513 - sigmoid_22_loss: 0.0509 - sigmoid_23_loss: 0.0395 - sigmoid_24_loss: 0.0593 - sigmoid_25_loss: 0.0516 - sigmoid_26_loss: 0.0554 - sigmoid_27_loss: 0.0452 - sigmoid_28_loss: 0.0506 - sigmoid_29_loss: 0.0315 - sigmoid_30_loss: 0.0527 - sigmoid_31_loss: 0.0519 - sigmoid_32_loss: 0.0200 - sigmoid_33_loss: 0.0104 - sigmoid_34_loss: 0.0040 - sigmoid_35_loss: 

    500/1721192 [..............................] - ETA: 56:46 - loss: 3.5104 - sigmoid_0_loss: 0.1548 - sigmoid_1_loss: 0.1961 - sigmoid_2_loss: 0.1810 - sigmoid_3_loss: 0.1199 - sigmoid_4_loss: 0.1212 - sigmoid_5_loss: 0.1813 - sigmoid_6_loss: 0.1161 - sigmoid_7_loss: 0.0915 - sigmoid_8_loss: 0.0926 - sigmoid_9_loss: 0.0466 - sigmoid_10_loss: 0.0984 - sigmoid_11_loss: 0.1071 - sigmoid_12_loss: 0.0749 - sigmoid_13_loss: 0.0535 - sigmoid_14_loss: 0.0066 - sigmoid_15_loss: 0.0592 - sigmoid_16_loss: 0.0541 - sigmoid_17_loss: 0.0631 - sigmoid_18_loss: 0.0935 - sigmoid_19_loss: 0.0492 - sigmoid_20_loss: 0.0759 - sigmoid_21_loss: 0.0445 - sigmoid_22_loss: 0.0475 - sigmoid_23_loss: 0.0398 - sigmoid_24_loss: 0.0759 - sigmoid_25_loss: 0.0160 - sigmoid_26_loss: 0.0643 - sigmoid_27_loss: 0.0886 - sigmoid_28_loss: 0.0962 - sigmoid_29_loss: 0.0149 - sigmoid_30_loss: 0.0454 - sigmoid_31_loss: 0.0236 - sigmoid_32_loss: 0.0101 - sigmoid_33_loss: 0.0058 - sigmoid_34_loss: 5.2979e-04 - sigmoid_35_loss: 

1721192/1721192 [==============================] - 3626s 2ms/step - loss: 3.5224 - sigmoid_0_loss: 0.1884 - sigmoid_1_loss: 0.1834 - sigmoid_2_loss: 0.1768 - sigmoid_3_loss: 0.1103 - sigmoid_4_loss: 0.1173 - sigmoid_5_loss: 0.1489 - sigmoid_6_loss: 0.1358 - sigmoid_7_loss: 0.0938 - sigmoid_8_loss: 0.0904 - sigmoid_9_loss: 0.0692 - sigmoid_10_loss: 0.0972 - sigmoid_11_loss: 0.1064 - sigmoid_12_loss: 0.0963 - sigmoid_13_loss: 0.0525 - sigmoid_14_loss: 0.0327 - sigmoid_15_loss: 0.0737 - sigmoid_16_loss: 0.0682 - sigmoid_17_loss: 0.0712 - sigmoid_18_loss: 0.0635 - sigmoid_19_loss: 0.0733 - sigmoid_20_loss: 0.0705 - sigmoid_21_loss: 0.0498 - sigmoid_22_loss: 0.0495 - sigmoid_23_loss: 0.0380 - sigmoid_24_loss: 0.0583 - sigmoid_25_loss: 0.0505 - sigmoid_26_loss: 0.0545 - sigmoid_27_loss: 0.0441 - sigmoid_28_loss: 0.0498 - sigmoid_29_loss: 0.0302 - sigmoid_30_loss: 0.0512 - sigmoid_31_loss: 0.0509 - sigmoid_32_loss: 0.0193 - sigmoid_33_loss: 0.0102 - sigmoid_34_loss: 0.0038 - sigmoid_35_loss: 

    500/1721192 [..............................] - ETA: 57:05 - loss: 3.3260 - sigmoid_0_loss: 0.1599 - sigmoid_1_loss: 0.2313 - sigmoid_2_loss: 0.1239 - sigmoid_3_loss: 0.0911 - sigmoid_4_loss: 0.1069 - sigmoid_5_loss: 0.1229 - sigmoid_6_loss: 0.1184 - sigmoid_7_loss: 0.0931 - sigmoid_8_loss: 0.0855 - sigmoid_9_loss: 0.0703 - sigmoid_10_loss: 0.0701 - sigmoid_11_loss: 0.1362 - sigmoid_12_loss: 0.0923 - sigmoid_13_loss: 0.0485 - sigmoid_14_loss: 0.0258 - sigmoid_15_loss: 0.0381 - sigmoid_16_loss: 0.0294 - sigmoid_17_loss: 0.0540 - sigmoid_18_loss: 0.0506 - sigmoid_19_loss: 0.0545 - sigmoid_20_loss: 0.0495 - sigmoid_21_loss: 0.0648 - sigmoid_22_loss: 0.0622 - sigmoid_23_loss: 0.0425 - sigmoid_24_loss: 0.0494 - sigmoid_25_loss: 0.0488 - sigmoid_26_loss: 0.0592 - sigmoid_27_loss: 0.0463 - sigmoid_28_loss: 0.0400 - sigmoid_29_loss: 0.0194 - sigmoid_30_loss: 0.0257 - sigmoid_31_loss: 0.0535 - sigmoid_32_loss: 0.0150 - sigmoid_33_loss: 0.0098 - sigmoid_34_loss: 0.0027 - sigmoid_35_loss: 0.03

1721192/1721192 [==============================] - 3690s 2ms/step - loss: 3.4497 - sigmoid_0_loss: 0.1835 - sigmoid_1_loss: 0.1797 - sigmoid_2_loss: 0.1734 - sigmoid_3_loss: 0.1072 - sigmoid_4_loss: 0.1140 - sigmoid_5_loss: 0.1458 - sigmoid_6_loss: 0.1330 - sigmoid_7_loss: 0.0912 - sigmoid_8_loss: 0.0871 - sigmoid_9_loss: 0.0667 - sigmoid_10_loss: 0.0946 - sigmoid_11_loss: 0.1043 - sigmoid_12_loss: 0.0943 - sigmoid_13_loss: 0.0504 - sigmoid_14_loss: 0.0317 - sigmoid_15_loss: 0.0720 - sigmoid_16_loss: 0.0661 - sigmoid_17_loss: 0.0692 - sigmoid_18_loss: 0.0618 - sigmoid_19_loss: 0.0714 - sigmoid_20_loss: 0.0694 - sigmoid_21_loss: 0.0483 - sigmoid_22_loss: 0.0479 - sigmoid_23_loss: 0.0366 - sigmoid_24_loss: 0.0569 - sigmoid_25_loss: 0.0491 - sigmoid_26_loss: 0.0535 - sigmoid_27_loss: 0.0428 - sigmoid_28_loss: 0.0486 - sigmoid_29_loss: 0.0291 - sigmoid_30_loss: 0.0498 - sigmoid_31_loss: 0.0496 - sigmoid_32_loss: 0.0186 - sigmoid_33_loss: 0.0100 - sigmoid_34_loss: 0.0035 - sigmoid_35_loss: 